In [4]:
import pandas as pd
import json
import os
from tqdm import tqdm
import ast
import shutil
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
source = "E:\Influencer dataset\Post metadata (JSON files)\info" 

In [3]:
time = '___noangel-1635867223678332074.info'

In [10]:
with open(source + '\___noangel-1635867223678332074.info') as f:
    data = json.load(f)

In [12]:
data['id']

'1635867223678332074'

# Ile osób znajduje się na zdjęciu - operacja na metadanych zbioru

In [15]:
data['accessibility_caption']

'Image may contain: 2 people'

In [14]:
abspath_src_folder_info = 'E:/Influencer dataset/Post metadata (JSON files)/info/'

In [20]:
for filename in tqdm(os.listdir(abspath_src_folder_info)):
    author_post = filename.split('-')[0]
    post_id = filename.split('-')[1].split('.')[0]
    data = None

    if os.path.getsize(abspath_src_folder_info + '/' + f"{filename}") == 0:
        continue

    with open(abspath_src_folder_info + '/' + f"{filename}") as f:
        data = json.load(f)
        f.close()
    
    if 'accessibility_caption' in data:
        record = {
            'post_author': author_post,
            'post_id': post_id,
            'count_persons': data['accessibility_caption']
        }
    else:
        record = {
            'post_author': author_post,
            'post_id': post_id,
            'count_persons': 'No info'
        }

    with open('E:\meta_count_person.json', 'a') as file:
        json.dump(record, file)
        file.write('\n')

100%|██████████| 97481/97481 [07:13<00:00, 224.76it/s]


In [71]:
def load_dictlist(f):
    ret = []
    for line in f:
        newdict = ast.literal_eval(line)
        if type(newdict) == dict:
            ret.append(newdict)
    return ret

In [72]:
with open('E:\meta_count_person.json', 'r') as file:
    data = [json.loads(line) for line in file]

df_person = pd.DataFrame(data)

In [73]:
df_person = df_person.drop_duplicates().reset_index(drop=True)

In [74]:
df_person

,post_author,post_id,count_persons
0,21andhungry,1666253905100487663,Image may contain: 1 person
1,21andhungry,1668268760837796181,Image may contain: 1 person
2,21andhungry,1669072952493961511,No photo description available.
3,21andhungry,1669793074191906720,No photo description available.
4,21andhungry,1670554832695241323,No photo description available.
...,...,...,...
97484,___noangel,2030225987314001912,"Image may contain: 1 person, closeup"
97485,___noangel,2030967648977663153,No info
97486,___noangel,2031614124434951959,"Image may contain: 1 person, closeup"
97487,___noangel,2033752253400951154,No info


In [75]:
df_person['count_persons'].value_counts()

No photo description available.                                                      16182
No info                                                                              10753
Image may contain: 1 person                                                          10299
Image may contain: food                                                               3070
Image may contain: 2 people                                                           2652
                                                                                     ...  
Image may contain: people sitting, plant, flower, table, tree, outdoor and nature        1
Image may contain: 2 people, wedding, ocean and outdoor                                  1
Image may contain: 5 people, wedding, tree and outdoor                                   1
Image may contain: flower and wedding                                                    1
Image may contain: 1 person, standing, hat and closeup                                   1

In [76]:
len(df_person['count_persons'].value_counts())

8377

In [2]:
def select(string):
    '''
    Utworzenie kategorii zaiwerjacące informacji dotyczące ilości osób na zdjęciu
    '''
    
    if 'person' in string:
        return '1 person'
    elif 'people' in string:
        return 'More than 1 person'
    elif ('No info' in string) or ('No photo description available.' in string):
        return 'No info'
    else:
        return 'Other'

In [78]:
for item in ['dsfdsf person','sdf sd people dsf sdf','No info ','No photo description available.','1 person, standing, hat and closeup','hat and closeup']:
    print(select(item))

1 person
More than 1 person
No info
No info
1 person
Other


In [79]:
df_person['class'] = df_person['count_persons'].apply(lambda x: select(str(x)))

In [80]:
df_person['class'].value_counts()

1 person              32926
No info               26935
More than 1 person    21503
Other                 16125
Name: class, dtype: int64

In [81]:
df_person['name'] = df_person['post_author'] + '-' + df_person['post_id'] + '.jpg'

In [83]:
df_person.head()

,post_author,post_id,count_persons,class,name
0,21andhungry,1666253905100487663,Image may contain: 1 person,1 person,21andhungry-1666253905100487663.jpg
1,21andhungry,1668268760837796181,Image may contain: 1 person,1 person,21andhungry-1668268760837796181.jpg
2,21andhungry,1669072952493961511,No photo description available.,No info,21andhungry-1669072952493961511.jpg
3,21andhungry,1669793074191906720,No photo description available.,No info,21andhungry-1669793074191906720.jpg
4,21andhungry,1670554832695241323,No photo description available.,No info,21andhungry-1670554832695241323.jpg


In [84]:
list_class = list(df_person['class'].unique())

In [90]:
for item in list_class:
    list_name = list(df_person[df_person['class'] == item]['name'].unique())
    print(len(list_name))

32926
26934
16125
21503


# Stworzenie odpowiednich folderów z podziałem osób

In [ ]:
abspath_src_folder_image = 'E:/Influencer dataset/Post images (JPG files)/posts_image/image/'
dest_folder = 'E:/'

In [ ]:
def splitting_images(abspath_src_folder_image,
                          dest_folder,
                          df_person):

    abspath_src_folder_image = os.path.abspath(abspath_src_folder_image)
    
    list_class = list(df_person['class'].unique())
    for item in list_class:
        list_name = list(df_person[df_person['class'] == item]['name'].unique())
        
        image = f"Influencer_dataset_sort_sort_images/Post images (JPG files)/posts_image/image/{item}"
        
        dest_folder_image = os.path.abspath(os.path.join(dest_folder, image))
        
        
        if not os.path.exists(dest_folder_image):
            os.makedirs(dest_folder_image)
            print(f"Utworzono folder: {dest_folder_image}")
        else:
            print(f"Folder {dest_folder_image} już istnieje.")
            
        print("Kopiowanie zdjęć")
        
        for filename in tqdm(os.listdir(abspath_src_folder_image)):
            for name in list_name:
                if filename.startswith(name):

                    src_file = os.path.join(abspath_src_folder_image, filename)
                    dest_file = os.path.join(dest_folder_image, filename)

                    shutil.copy(src_file,dest_file)
                    break

In [ ]:
splitting_images(abspath_src_folder_image,
                 dest_folder,
                 df_person)                 